# Image File Converter 

This notebook supports the following file types:
*   Input: png, jpg, tiff, mrc
*   Output: png, jpg, tiff

When converting mrc files, it may be useful to visualize the original file in napari if you want the file to save as it is displayed in napari.

A gentle reminder: please upload the image you want to convert to JupyterHub before running this notebook. The converted image will also appear in JupyterHub and can be downloaded onto your computer.

### Install and import dependencies:

In [ ]:
import sys
!{sys.executable} -m pip install pathlib mrcfile Pillow numpy tifffile 

import pathlib as pl
import mrcfile
from PIL import Image as im
import numpy as np
import tifffile as tif

### Import your file and set conversion parameters:

Note: 'jpeg' and 'tif' will not work as inputs for the destination file type. Please be mindful of this, and any typos, as the code is written to exit the program if it takes an unknown input.

In [ ]:
#import file
filename = input('File name:')
filetype = pl.Path(filename).suffix

# check input filetype
valid_filetypes = ['.png', '.tif', '.tiff', '.mrc', '.jpg', '.jpeg']

if filetype in valid_filetypes:
    pass
else:
    print('File type not accepted.')
    sys.exit()

#specify destination filetype
to_file = input('Convert to (png/tiff/jpg):')

def suffix(to_file):
    if to_file == 'png':
        return '.png'
    elif to_file == 'tiff':
        return '.tiff'
    elif to_file == 'jpg':
        return '.jpg'
    else:
        print('File type not accepted. Please try again.')
        sys.exit()

new_name = input('New file name:') + suffix(to_file)

### The image is converted to a numpy array:

In [ ]:
def img_to_array(filename):
    if filetype == '.mrc': 
        return mrcfile.mmap(filename, permissive=True).data
    else:
        return np.asarray(im.open(filename))

array = img_to_array(filename)

### Set contrast limits. Details in the code:

In [ ]:
# to find the min and max values of the array, if necessary
print(np.min(array))
print(np.max(array))

In [ ]:
#set contrast limits
if filetype == '.mrc':
    contrast_min = array.min() # or a specified minimum pixel value
    contrast_max = array.max() # or a specified maximum pixel value
    
    ## contrast limits of mrc files must be manually adjusted to appear as they look in napari. in rare cases, 
    ## the limits are equal to the min and max of the image. if so, use the same parameters for all other image 
    ## file types. napari does not seem to adjust the contrast limits in any substantial way; these values may 
    ## be found in metadata associated with the image itself. for now, the only way to find these values is by 
    ## trial and error. it may be helpful to find the min and max values using the block above. contrast limits 
    ## are often somewhere in the middle of that range.

else:
    contrast_min = array.min()
    contrast_max = array.max()

array = np.where(array <= contrast_min, contrast_min, array)
array = np.where(array >= contrast_max, contrast_max, array)
array = (array-np.min(array))/(np.max(array)-np.min(array))

if to_file == 'tiff':
    pass
else:
    array *= 255/array.max()

### Convert the array to the desired image file and return output:

In [ ]:
#convert array to new image filetype
def array_to_img(array):
    if to_file == 'tiff':
        array = array.astype(np.float32)
        return tif.imwrite(new_name, array)
    elif to_file == 'png':
        array = array.astype(np.uint8)
        png = im.fromarray(array)
        return png.save(new_name, Format='PNG')
    else:
        array = array.astype(np.uint8)
        jpg = im.fromarray(array).convert('RGB')
        return jpg.save(new_name, Format='JPG')

array_to_img(array)
print(pl.Path(filename).name, ' successfully converted to ', to_file,', see file ', new_name, sep='')